# **Recommender Systems**

<img src="../images/recsys-posters.png">

<a href="https://colab.research.google.com/github/deep-learning-indaba/indaba-pracs-2024/blob/main/practicals/Recommender_Systems.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

© Deep Learning Indaba 2023. Apache License 2.0.

**Authors:** [Amrit Purshotam](https://nl.linkedin.com/in/amritpurshotam)

**Introduction:**

Recommender Systems are probably one of the most ubiquitous type of machine learning model that we encounter in our online life. They influence what we see in our social media feeds, the products we buy, the music we listen to, the food we eat, and the movies we watch. Sometimes they're so good that people feel that their phone is spying on their conversations! In this prac, we hope to convince you that this isn't the case (mostly) as well as taking you through some of the techniques popularly used in industry that recommends the content you see online by building our very own movie recommender system.

**Topics:**

Content: Machine Learning, Recommender Systems, Approximate Nearest Neighbours

**Aims/Learning Objectives:**

- General architecture of a recommender systems.
- Techniques for making recommendations.
- Serving recommendations efficiently in production.

**Before you start:**

For this practical, you will need to use a GPU to speed up training. To do this, go to the "Runtime" menu in Colab, select "Change runtime type" and then in the popup menu, choose "GPU" in the "Hardware accelerator" box.

## Installation and Imports

In [ ]:
## Install and import anything required. Capture hides the output from the cell.
# @title Import required packages. (Run Cell)

import os
import sys
import zipfile
from typing import Iterable, Mapping, Tuple, Sequence
from textwrap import wrap

import requests
import jax
import optax
import numpy as np
import pandas as pd
import tensorflow as tf
from flax import struct
from clu import metrics
from flax import linen as nn
from jax import numpy as jnp
import jax.tree_util as tree
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
from flax.training import train_state

SEED = 42
np.random.seed(SEED)

In [ ]:
# @title Helper methods

def download_posters():
    #url = "https://drive.google.com/uc?id=15ipM74Qdc74d25jRQ3Mf4yB79tZqSkjU"
    url = "https://drive.usercontent.google.com/download?id=15ipM74Qdc74d25jRQ3Mf4yB79tZqSkjU&authuser=0&confirm=t"
    response = requests.get(url, stream=True)
    with open("../data/posters.zip", mode="wb") as file:
        for chunk in response.iter_content(chunk_size=10*1024):
            file.write(chunk)

def unzip_posters():
    with zipfile.ZipFile("../data/posters.zip", 'r') as zip_ref:
        zip_ref.extractall("../data/")

def get_dataset(ds_name: str) -> pd.DataFrame:
  ds, info = tfds.load(f'movielens/{ds_name}', data_dir="../data", with_info=True)
  df = tfds.as_dataframe(ds['train'], info)
  df = df.astype({'user_id': int, 'movie_id': int})
  df.loc[:, 'movie_title'] = df['movie_title'].str.decode("utf-8")
  return df

def cross_tabulate(df: pd.DataFrame, num_samples: int = 10) -> pd.DataFrame:
  pivot_df = df.pivot(index='user_id', columns='movie_id', values='user_rating')
  pivot_df = pivot_df.loc[df['user_id'].sample(num_samples), df['movie_id'].sample(num_samples)].dropna(axis=0, thresh=1).fillna("")
  return pivot_df

def make_mapping(id_set: Iterable[str]) -> Mapping[str, int]:
  return {id_str: i for (i, id_str) in enumerate(id_set)}

def densify_column_values(df: pd.DataFrame, col_name: str) -> Tuple[pd.Series, Sequence[str]]:
  col_values = sorted(set(df[col_name]))
  col_ids_map = make_mapping(col_values)
  return df[col_name].apply(lambda col_id: col_ids_map[col_id]), col_values, col_ids_map

def to_tfds(df: pd.DataFrame) -> tf.data.Dataset:
  fields = {
    ('user_id', tf.int32),
    ('item_id', tf.int32),
    ('user_rating', tf.float32),
    ('timestamp', tf.int32)
  }

  tensor_slices = {
      field: tf.cast(df[field].values, dtype=field_type)
      for field, field_type in fields
  }

  return tf.data.Dataset.from_tensor_slices(tensor_slices)

def get_train_val_split(df: pd.DataFrame):
  val_df = df.sample(frac=0.2)
  train_df = df[~df.index.isin(val_df.index)]

  return train_df, val_df

def prepare_dataloaders(train_df: pd.DataFrame, val_df: pd.DataFrame, batch_size: int, num_epochs: int):
  train_ds = (
    to_tfds(train_df)
    .repeat(config.NUM_EPOCHS)
    .shuffle(1024)
    .batch(config.BATCH_SIZE, drop_remainder=False)
    .prefetch(1)
  )
  val_ds = (
    to_tfds(val_df)
    .shuffle(1024)
    .batch(config.BATCH_SIZE, drop_remainder=False)
    .prefetch(1)
  )
  return train_ds, val_ds

def plot_metric_history(metrics_history):
  fig, ax1 = plt.subplots(1, 1, figsize=(7, 5))
  ax1.set_title('Loss')
  for dataset in ('train','val'):
    ax1.plot(metrics_history[f'{dataset}_loss'], label=f'{dataset}_loss')
  ax1.legend()
  plt.show()
  plt.clf()

def make_movie_lookup(df: pd.DataFrame):
    movies = df[['movie_id', 'movie_title']].drop_duplicates(subset='movie_id')
    movie_lookup = dict(zip(movies.movie_id, movies.movie_title))
    return movie_lookup

In [ ]:
# @title Config

class Config:
    DATASET = "latest-small-ratings" # all the options are 100k-ratings 1m-ratings 20m-ratings 25m-ratings latest-small-ratings
    SEED = 42
    EMB_DIM = 50
    LR = 5e-3
    BATCH_SIZE = 64
    NUM_EPOCHS = 5
    USE_POSTERS = True

config = Config()

rng = jax.random.PRNGKey(config.SEED)
rng, model_rng, dataset_rng = jax.random.split(rng, 3)

df = get_dataset(config.DATASET)

if config.USE_POSTERS:
    download_posters()
    unzip_posters()

## **1. Recommender Systems Overview**

### **1.1 A Real World Scenario**

Imagine you're going shopping for a new book. You enter the store and start walking around scanning the shelves of books. Something catches your interest, you pause, take the book down and inspect the cover, and perhaps also read the blurb. You continue to do this until you find something you like at which point you purchase the book and leave.

Now imagine you've read the book and quite enjoyed it and you want to buy another one. You go back to the store, browse around, and occasionally inspect some books that catch your interest. An assistant this time approaches you asking if you need help. Gladly you accept, and you mention the books you were looking at as well as the one that you purchased recently. Since they've been working there a long time and have helped many customers, they now have a good idea of what you may like and recommends a short list of books for you to look at. You do so and eventually settle on one to purchase.

**Exercise**. Let's pause here for a moment and dig deeper into this scenario.

- From all the books in the store, what does it say about the ones you paused to look at, the ones you ignored, and the ones you purchased?
- What does it say about you as the reader and your preferences? Could there be
other people like you?
- How was the assistant able to narrow down all the books in the store to just a few from which you actually purchased one?

<img src="../images/book_store.png" />

Source: Kim Falk. *Practical Recommender Systems*. 2019. Manning.

The answer to these questions is now getting into the heart of recommender systems. Your behaviour probably wasn't random but instead had some structure and logic to it. You also likely have particular preferences to some genres and/or authors. Without even knowing anything else about the books, this follows then that the books you looked at and purchased matched those preferences and the ones you ignored more likely did not. Additionally, based on your preferences, what was catching your interest, and what you previously read, the store assistant was able to stitch together a rough profile of you. She then thought about her previous customers similar to you and what they had previously purchased. This is how she was then able to shortlist relevant books for you to look at.

As you probably guessed it by now, the store assistant is the recommender system in this example. Instead of a person though, we want to build something that is able to learn the latent structure between all the books, all our customers, and how well they match each other in order to then make our recommendations. In the following sections we will learn how to do this. But first let's go over the general architecture of a recommender system.

### **1.2 Recommendation System Architecture**

#### **1.2.1 Overview**
![picture](../images/recsys_overview.png)
Source: [*Recommendation System Design*](https://medium.com/double-pointer/system-design-interview-recommendation-system-design-as-used-by-youtube-netflix-etc-c457aaec3ab). 2021. Medium.

The above diagram generalises our previous scenario from books to searches, songs, and movies (note there will usually only be one) while the user is you as before. Notice how the user interacts with these *items* which then gets sent to the recommender system as feedback. The recommender system processes this, retrieves relevant items from it's database and then serves them to the user. The user, in turn, further interacts with these items and continues this loop until some terminating criteria. Perhaps they found a movie they like and started watching, or they purchase a book like in our previous example, or they simply leave.

Finally, you may have noticed some new terminology in the diagram, the *query / context* on the user side of diagram. This is a further generalisation of what we're retrieving relevant items for. In this example it's the user but it can also include their previous searches, item interactions, the time of day, the device they're using, or any of combination of them.

#### **1.2.2 Zooming in**

Let's zoom in closer to the recommender system now by having a look at how YouTube described theirs back in 2016 in their seminal paper on the topic. Being YouTube, the items here would be videos which number in the millions (and most likely in the billions as of writing in 2023). Pay special attention to the blue stages, notice the number of items going into each stage going down and hence the funnel shape.

Looking at the first one, the job of the *candidate generation* stage is to efficiently *retrieve* relevant items from your database (which is why you may find in the literature, this stage is also known as *retrieval*). Speed is of the utmost importance here so some leeway is allowed in terms of relevancy as long as the number of items we reduce down to is manageable for the downstream parts of the recommender. This lookup speed is achieved partly by the techniques we discuss and implement later in this practical but also by limiting the number of features that feed into this stage.

The second *ranking* stage (also known as *scoring*) then takes these items and sorts them based on additional features that come from the user as well as the features of the item itself to optimise for some target we care about. In the case of YouTube, it will be for watch time, or in the case of an e-commerce website, the likelihood to purchase the items. Another reason for a ranking stage is that you may have multiple candidate generators and you now need a way to combine the results in some optimal way, at which point they're then shown to the user. It's also important to note, this ranking stage is not always necessary. Sometimes the retrieval step is good enough and you can keep the complexity of the system down, reduce implementation times, lower maintenance overhead, and therefore costs.

![picture](../images/recsys_yt.png)

Source: Covington et. al. [*Deep Neural Networks for YouTube Recommendations*](https://research.google/pubs/pub45530/). 2016. Proceedings of the 10th ACM Conference on Recommender Systems.

Finally, this last diagram, courtesy of the recommendations team at NVIDIA, further expands on the above ideas by defining two more stages namely *filtering* and *ordering*. After the retrieval / candidate generation stage, we may find that some of the items, while relevant, aren't useful and so need to be filtered out. In an e-commerce scenario this could be an item that's out of stock or in the case of a social media platform, a post coming from a person or topic you've blocked / muted. The *ordering* step which takes place after ranking / scoring then refers to further refining the order of the items depending on some business logic. For example, promoting on sale items or perhaps even sponsored placements.

![picture](../images/recsys_architecture.png)

Source: Even Oldridge. [*Recommender Systems, Not Just Recommender Models*](https://medium.com/nvidia-merlin/recommender-systems-not-just-recommender-models-485c161c755e). 2022. Medium.

The rest of this practical will now focus primarily on the retrieval / candidate generation stage of a recommender system. We hope this introduction and overview of recommender systems helps put into context the specific piece we will be building out. In particular, we will be learning about Collaborative Filtering and Graph Neural Networks for recommendations.

## **2. Collaborative Filtering**

### **2.1 Overview**

Collaborative Filtering is a technique that learns how to recommend items to user A based on the interests of a similar user B by using similarities between these users and items simultaneously. This is opposed to content-based filtering that uses hand-engineered and/or explicit features of an item for further recommendations for e.g. recommending a movie in the same genre of one who you already watched.

<img src="../images/cbcf.png" width="100%">

Source: Arthur Mello. [*How do Netflix and Amazon know what I want?*](https://towardsdatascience.com/how-do-netflix-and-amazon-know-what-i-want-852c480b67ac). 2020. Medium.

### **2.2 Matrix Factorisation**

Consider a movie recommendation system in which the training data consists of a feedback matrix $A ∈ R^{m × n}$ where $m$ is the number users and $n$ is the number of items. Each row then represents a user and each column represents an item (a movie) with the entries indicating a users rating of a particular movie. Our goal then is to learn
- A user embedding matrix $U ∈ \mathbb{R}^{m × d}$, where row $i$ is the embedding for user $i$ and $d$ is the length of the embedding.
- An item embedding matrix $V ∈ \mathbb{R}^{n × d}$, where row $j$ is the embedding for item $j$ and $d$ is the length of the embedding.

such that their product $UV^{T}$ is a good approximation of the feedback matrix $A$. Note that the $(i, j)$ entry of $UV^{T}$ is simply the dot product of $⟨ U_i, V_j ⟩$ of the embeddings of user $i$ and item $j$ which you want to be as close as possible to $A_{ij}$. This process of finding these matrices $U$ and $V$ is known as *matrix factorisation*.

<img src="../images/MatrixFactor.svg" width="100%" />


Source: [*Google Recommendation Systems Course*](https://developers.google.com/machine-learning/recommendation/collaborative/matrix).

Take special note how the matrices $U$ and $V$ give a more compact representation of the full matrix, even in this toy example (in practice $d$ is much smaller than $m$ and $n$ since you will have many more users and items). As a result, this process finds latent structure in the data without even requiring any knowledge of the movies themselves!

Now, there are two common algorithms for finding these matrices, namely
- Stochastic Gradient Descent (SGD) which is a generic method to minimise loss functions
- Weighted Alternating Least Squares (WALS) which is specialised to this particular problem and works by alternating between fixing $U$ and solving for $V$ and vice versa.

For this practical, we'll be using SGD to learn these matrices which means we have to define a loss function. One intuitive function is the squared distance. To do this, we minimise the average of squared errors over all pairs of observed entries.

$ \frac{1}{|obs|} ∑_{(i,j) \in obs} (A_{ij} - ⟨ U_i, V_j ⟩)^2$

With this loss function defined, we can then randomly initialise our matrices $U$ and $V$, and through SGD iteratively update them until $UV^T$ is a good approximation of $A$. We now finally have all the pieces to move ahead with the rest of the tutorial.

### **2.3 The MovieLens Dataset**

Since we don't have access to an actual streaming service watch history, we will instead use a dataset called [MovieLens](https://grouplens.org/datasets/movielens/) and in particular the `latest-small-ratings` subset which contains 100 thousand ratings across 9 thousand movies and 600 users. Let's have a look.

In [ ]:
df = get_dataset(config.DATASET)
movies = df[['movie_id', 'movie_title']].drop_duplicates()
movie_lookup = make_movie_lookup(df)
df.head(5)[['user_id', 'movie_title', 'user_rating', 'timestamp']]

As you can see, we have our users represented by the `user_id`, the movies (our items), and the `user_rating` out of 5 that the user gave that particular movie. We also have the `timestamp` of when the user rated the movie in unix time format which is the number of seconds that elapsed since 1 January 1970 UTC and a common way of representing time due to it's unambiguity.

Now what if we cross-tabulate a sample of this data to get an alternative view.

In [ ]:
cross_tabulate(df)

The table displayed shows some of the more popular movies and users. The empty cells are what we want our model to learn to fill in i.e. the movies we presume a user has not yet watched because they have yet to rate it. Then once we make these predictions, we can figure out which of those movies they're most likely to enjoy.

Looking at this table, you may have also noticed it's sparsity. In reality, this table is actually even more so. Let's calculate how many cells are empty.

In [ ]:
num_users = df['user_id'].unique().shape[0]
num_items = df['movie_title'].unique().shape[0]
num_ratings = df.shape[0]
sparsity = 100 - (num_ratings / (num_users * num_items) * 100)
print(f"Sparsity: {sparsity:.2f}")



### **2.2 Dataset Preparation**

Now let's prepare our dataset for training. We will be mapping our users and movies from indexes starting from 0.

In [ ]:
df['user_id'], user_list, user_to_id_mapping = densify_column_values(df, 'user_id')
df['item_id'], movie_list, movie_to_id_mapping = densify_column_values(df, 'movie_title')
df.head(5)[['user_id', 'item_id', 'user_rating', 'timestamp']]

Then we will split our data randomly into a train and validation set and create our dataloaders that will feed data into our training process later. The exact details aren't too important here but if you're interested feel free to inspect these methods in the Helper methods cell near the top of this notebook.

In [ ]:
train_df, val_df = get_train_val_split(df)
train_ds, val_ds = prepare_dataloaders(train_df, val_df, config.BATCH_SIZE, config.NUM_EPOCHS)
total_steps = train_ds.cardinality().numpy()

We are now ready to finally start implementing our Collaborative Filtering model.

### **2.3 Building our Model**

#### **2.3.1 Dot Product Model**

Let's start defining the architecture for this model in it's simplest formulation. Implement the below steps in the code cell below.
1. Define $m$ and $n$ which correspond to the number of users and items respectively.
2. Define $d$ corresponding to the dimension of the below embedding matrices.
3. Create and specify the shape of our embedding matrices. Recall $U ∈ \mathbb{R}^{m × d}$ and $V ∈ \mathbb{R}^{n × d}$ for the user and item embeddings respectively. Hint: [`nn.Embed`](https://flax.readthedocs.io/en/latest/api_reference/flax.linen/_autosummary/flax.linen.Embed.html)
4. Take the dot product of the user and item embeddings respectively. Hint: [`jnp.multiply`](https://jax.readthedocs.io/en/latest/_autosummary/jax.numpy.multiply.html) and [`jnp.sum`](https://jax.readthedocs.io/en/latest/_autosummary/jax.numpy.sum.html) may be useful here.

In [ ]:
class CFDotProduct(nn.Module):
  # Step 1 and 2
  # define the number of users (corresponds to m)
  # define the number of items (corresponds to n)
  # define the embedding dimension here (corresponds to d)

  @nn.compact
  def __call__(self, x):
    # Step 3 Specify the shape of the embedding matrices
    users = nn.Embed(num_embeddings=? features=?, name='user_embs')(x['user_id'])
    items = nn.Embed(num_embeddings=? features=?, name='item_embs')(x['item_id'])

    y = # Step 4 take the dot product of users and items
    return y

In [ ]:
# @title Solution
class CFDotProduct(nn.Module):
  num_users: int
  num_items: int
  emb_dim: int

  @nn.compact
  def __call__(self, x):
    users = nn.Embed(num_embeddings=self.num_users, features=self.emb_dim, embedding_init=nn.initializers.normal(), name='user_embs')(x['user_id'])
    items = nn.Embed(num_embeddings=self.num_items, features=self.emb_dim, embedding_init=nn.initializers.normal(), name='item_embs')(x['item_id'])

    y = jnp.sum(jnp.multiply(users, items), axis=1)
    return y

Now that we have specified the shape of the model, we now have to define our loss function. Recall that this will be the mean squared error between our predicted ratings and the actual ratings. Since this loss is a reasonable metric as well to measure the performance of our model, we'll use it as well. Hint: [`optax.squared_error`](https://optax.readthedocs.io/en/latest/api.html#optax.squared_error).

In [ ]:
@jax.jit
def train_step(state, batch):
  def loss_fn(params):
    logits = state.apply_fn(params, batch)
    loss = # fill in the loss function here and then take the mean from our batch
    return loss
  grad_fn = jax.grad(loss_fn)
  grads = grad_fn(state.params)
  state = state.apply_gradients(grads=grads)
  return state

@jax.jit
def compute_metrics(*, state, batch):
  logits = state.apply_fn(state.params, batch)
  loss = # fill in the loss function here and then take the mean from our batch
  metric_updates = state.metrics.single_from_model_output(logits=logits, labels=batch['user_rating'], loss=loss)
  metrics = state.metrics.merge(metric_updates)
  state = state.replace(metrics=metrics)
  return state

In [ ]:
# @title Solution
@jax.jit
def train_step(state, batch):
  def loss_fn(params):
    logits = state.apply_fn(params, batch)
    loss = optax.squared_error(logits, batch['user_rating']).mean()
    return loss
  grad_fn = jax.grad(loss_fn)
  grads = grad_fn(state.params)
  state = state.apply_gradients(grads=grads)
  return state

@jax.jit
def compute_metrics(*, state, batch):
  logits = state.apply_fn(state.params, batch)
  loss = optax.squared_error(logits, batch['user_rating']).mean()
  metric_updates = state.metrics.single_from_model_output(logits=logits, labels=batch['user_rating'], loss=loss)
  metrics = state.metrics.merge(metric_updates)
  state = state.replace(metrics=metrics)
  return state

Now that we have our model and loss functions defined, let's initialise our training state and define our training loop.

In [ ]:
@struct.dataclass
class Metrics(metrics.Collection):
    loss: metrics.Average.from_output('loss')

class TrainState(train_state.TrainState):
    metrics: Metrics

def initialise_train_state(model, model_rng, dataloader, total_steps, weight_decay=None):
  params = model.init(model_rng, next(tfds.as_numpy(dataloader).__iter__()))
  scheduler = optax.cosine_onecycle_schedule(
    transition_steps=total_steps,
    peak_value=config.LR,
    pct_start=0.25,
    div_factor=25.0,
    final_div_factor=100000.0
  )

  if weight_decay:
    tx = optax.adamw(learning_rate=scheduler, weight_decay=weight_decay)
  else:
    tx = optax.adam(learning_rate=scheduler)

  state = TrainState.create(
    apply_fn=model.apply,
    params=params,
    tx=tx,
    metrics=Metrics.empty()
  )
  return state

def train_model(state, train_ds, val_ds):
  num_steps_per_epoch = train_ds.cardinality().numpy() // config.NUM_EPOCHS

  metrics_history = {
    'train_loss': [],
    'val_loss': [],
  }

  for step, batch in enumerate(train_ds.as_numpy_iterator()):
    state = train_step(state, batch)
    state = compute_metrics(state=state, batch=batch)

    if (step+1) % num_steps_per_epoch == 0:
      for metric, value in state.metrics.compute().items():
          metrics_history[f'train_{metric}'].append(value)
      state = state.replace(metrics=state.metrics.empty())

      val_state = state
      for val_batch in val_ds.as_numpy_iterator():
          val_state = compute_metrics(state=val_state, batch=val_batch)

      for metric, value in val_state.metrics.compute().items():
          metrics_history[f'val_{metric}'].append(value)

      print(
          f"train epoch: {(step+1) // num_steps_per_epoch}, "
          f"loss: {metrics_history['train_loss'][-1]}, "
      )
      print(
          f"val epoch: {(step+1) // num_steps_per_epoch}, "
          f"loss: {metrics_history['val_loss'][-1]}, "
      )

  return state, metrics_history

We are now ready to train our model!

In [ ]:
model = CFDotProduct(num_users=len(user_list), num_items=len(movie_list), emb_dim=config.EMB_DIM)
state = initialise_train_state(model, model_rng, train_ds, total_steps)
state, metric_history = train_model(state, train_ds, val_ds)
plot_metric_history(metric_history)

Since our ratings are between `0.5` and `5`, the first thing we can do to make this model a bit better is to force the predictions to be in this range. A useful function for this would be the [`sigmoid`](https://flax.readthedocs.io/en/v0.5.3/_autosummary/flax.linen.sigmoid.html) which constrains it's input to between `0` and `1`. Multiplying and shifting this then allows us to move these values to any range of values. Let's choose `0` and `5.5` since empirically these work better by expanding the range of allowed values.

In [ ]:
class CFWithRange(nn.Module):
  num_users: int
  num_items: int
  emb_dim: int
  # define the min_rating
  # define the max_rating

  @nn.compact
  def __call__(self, x):
    user_embeds = nn.Embed(num_embeddings=self.num_users, features=self.emb_dim, embedding_init=nn.initializers.normal(), name='user_embs')(x['user_id'])
    item_embeds = nn.Embed(num_embeddings=self.num_items, features=self.emb_dim, embedding_init=nn.initializers.normal(), name='item_embs')(x['item_id'])

    y = jnp.sum(jnp.multiply(user_embeds, item_embeds), axis=1)
    y = # constrain y to between 0 and 1
    y = # expand the range to between 0 and 5.5.
    return y

In [ ]:
# @title Solution
class CFWithRange(nn.Module):
  num_users: int
  num_items: int
  emb_dim: int
  min_rating: float
  max_rating: float

  @nn.compact
  def __call__(self, x):
    user_embeds = nn.Embed(num_embeddings=self.num_users, features=self.emb_dim, embedding_init=nn.initializers.normal(), name='user_embs')(x['user_id'])
    item_embeds = nn.Embed(num_embeddings=self.num_items, features=self.emb_dim, embedding_init=nn.initializers.normal(), name='item_embs')(x['item_id'])

    y = jnp.sum(jnp.multiply(user_embeds, item_embeds), axis=1)
    y = nn.sigmoid(y)
    y = y * (self.max_rating - self.min_rating) + self.min_rating
    return y

We're now ready to train the model again. Let's see how it does.

In [ ]:
model = CFWithRange(
  num_users=len(user_list),
  num_items=len(movie_list),
  emb_dim=config.EMB_DIM,
  min_rating=0,
  max_rating=5.5
)
state = initialise_train_state(model, model_rng, train_ds, total_steps)
state, metric_history = train_model(state, train_ds, val_ds)
plot_metric_history(metric_history)

If everything ran correctly, this should have performed a bit better. A missing piece now is that some users are just more positive or negative in their ratings than others, and some movies are just plain better or worse than others. But in our dot product representation we do not have any way to encode either of these things. If all you can say about a movie is, for instance, that it is very sci-fi or very action-oriented, then you don't really have any way to say whether most people like it.

That's because at this point we only have weights; we do not have biases. If we have a single number for each user that we can add to our scores, and the same for each movie, it will handle this missing piece. Let's adjust our model architecture again.

#### **2.3.2 Dot Product with Bias Model**

For each user and for each item, let's add a single number. This corresponds to one dimensional embeddings each the length of the number of users and number of items respectively. Let's add them below.

In [ ]:
class CFDotProductBias(nn.Module):
  num_items: int
  num_users: int
  emb_dim: int
  min_rating: float
  max_rating: float

  @nn.compact
  def __call__(self, x):
    user_embeds = nn.Embed(num_embeddings=self.num_users, features=self.emb_dim, embedding_init=nn.initializers.normal(), name='user_embs')(x['user_id'])
    item_embeds = nn.Embed(num_embeddings=self.num_items, features=self.emb_dim, embedding_init=nn.initializers.normal(), name='item_embs')(x['item_id'])
    # Add the 1 dimensional bias terms here
    user_bias = nn.Embed(num_embeddings=?, features=?, embedding_init=nn.initializers.normal(), name='user_bias')(x['user_id'])
    item_bias = nn.Embed(num_embeddings=?, features=?, embedding_init=nn.initializers.normal(), name='item_bias')(x['item_id'])

    y = jnp.sum(jnp.multiply(user_embeds, item_embeds), axis=1, keepdims=1)
    y += # add the biases
    y = nn.sigmoid(y) * (self.max_rating - self.min_rating) + self.min_rating
    return jnp.squeeze(y)

In [ ]:
# @title Solution
class CFDotProductBias(nn.Module):
  num_items: int
  num_users: int
  emb_dim: int
  min_rating: float
  max_rating: float

  @nn.compact
  def __call__(self, x):
    item_embeds = nn.Embed(num_embeddings=self.num_items, features=self.emb_dim, embedding_init=nn.initializers.normal(), name='item_embs')(x['item_id'])
    item_bias = nn.Embed(num_embeddings=self.num_items, features=1, embedding_init=nn.initializers.normal(), name='item_bias')(x['item_id'])
    user_embeds = nn.Embed(num_embeddings=self.num_users, features=self.emb_dim, embedding_init=nn.initializers.normal(), name='user_embs')(x['user_id'])
    user_bias = nn.Embed(num_embeddings=self.num_users, features=1, embedding_init=nn.initializers.normal(), name='user_bias')(x['user_id'])

    y = jnp.sum(jnp.multiply(user_embeds, item_embeds), axis=1, keepdims=1)
    y += user_bias + item_bias
    y = nn.sigmoid(y) * (self.max_rating - self.min_rating) + self.min_rating
    return jnp.squeeze(y)

Let's train our model and see how this one performs.

In [ ]:
model = CFDotProductBias(
  num_items=len(movie_list),
  num_users=len(user_list),
  emb_dim=config.EMB_DIM,
  min_rating=0.0,
  max_rating=5.5
)
state = initialise_train_state(model, model_rng, train_ds, total_steps)
state, metric_history = train_model(state, train_ds, val_ds)
plot_metric_history(metric_history)

##### **2.3.2.1 Mitigate overfitting**

It appears our model has started to overfit since our validation loss started getting worse while our training loss continued to decrease. One way to mitigate this is to constrain our model with weight decay. The details aren't important right now but the main idea is that this technique prevents our weights from getting too high, a symptom of overfitting. Luckily this time, we don't even need to change our model, we can specify this as a parameter in our state initialisation step and the details are taken care of for you. Let's try training our model again.

In [ ]:
model = CFDotProductBias(
  num_items=len(movie_list),
  num_users=len(user_list),
  emb_dim=config.EMB_DIM,
  min_rating=0.0,
  max_rating=5.5
)
state = initialise_train_state(model, model_rng, train_ds, total_steps, weight_decay=0.1)
state, metric_history = train_model(state, train_ds, val_ds)
plot_metric_history(metric_history)

Great, hopefully this should have led to an improved model and stabilised our validation loss. Now let's try to interpret the model we just built.

### **2.4 Interpreting our Model**

#### **2.4.1 Biases**

We mentioned earlier that the biases we added to the model would be able to encode whether some users are generally more positive or negative in their reviews and whether some movies are universally better or worse than others. Let's put this to the test by inspecting the biases. First we'll look at the worst movies i.e. the movies with the lowest biases





In [ ]:
item_movie_df = df.drop_duplicates(subset='movie_id')[['item_id', 'movie_id']]
item_id_to_movie_id_mapping = dict(zip(item_movie_df.item_id, item_movie_df.movie_id))

id_to_movie_mapping = {}
for movie, id in movie_to_id_mapping.items():
    id_to_movie_mapping[id] = movie

def display_recommendations(item_ids: list[int]):
    if config.USE_POSTERS:
        movie_ids = [item_id_to_movie_id_mapping[item_id] for item_id in item_ids]
        fig, ax = plt.subplots(nrows=1, ncols=5, figsize=(20, 7), dpi=80, sharex=True, sharey=True)
        for i, movie_id in enumerate(movie_ids):
            poster_path = f'../data/posters/{movie_id}.jpg'
            if not os.path.isfile(poster_path):
                poster_path = f'../data/posters/default.jpg'
            image = plt.imread(poster_path)
            ax[i].imshow(image)
            ax[i].axis('off')
            ax[i].set_title("\n".join(wrap(movie_lookup[movie_id], 35)))
    else:
        for item_id in item_ids:
            print(id_to_movie_mapping[item_id])

In [ ]:
item_ids = state.params['params']['item_bias']['embedding'].squeeze().argsort().tolist()[:5]
display_recommendations(item_ids)

Hopefully none of these are any of your favourites!

Let's now look at what the model learned to be some of the best movies.

In [ ]:
item_ids = state.params['params']['item_bias']['embedding'].squeeze().argsort().tolist()[-5:]
display_recommendations(item_ids)

Do you recognise any of these and if so, do you agree with the model that these movies are generally universally loved? If you haven't watched any of them, perhaps you can choose one of these the next time you're looking for something and put this model to the test!

#### **2.4.2 Embedding Distance**

Our embeddings aren't quite so easy to directly interpret since there's too many factors to look at. We can however measure similarity between them. More formally a similarity measure is a function $s : E × E → \mathbb{R}$ that takes a pair of embeddings and returns a scalar measuring their similarity. The embeddings can be used for candidate generation as follows: given a query embedding $q \in E$, the system looks for item embeddings $x \in E$ that are close to q i.e. embedding with high similarity $s(q,x)$. To determine similarity, most recommendation systems rely on one or more of the following:
 - **Cosine**: the cosine of the angle between the two vectors $s(q,x) = cos(q,x)$
 - **Dot Product**: the dot product of the two vectors $s(q,x) = ∑_{i=1}^{d} q_i x_i$
 - **Euclidean distance**: The distance in Euclidean space, $s(q,x) = ||q - x|| = [∑_{i=1}^{d} (q_i - x_i)^2]^{\frac{1}{2}}$

 For the purposes of this practical, let's choose cosine similarity for our recommender. We first define the function that will calculate the cosine similarity between our selected item embedding and all the other embeddings

In [ ]:
def get_cos_sim_distances(item, embeddings):
  dot_products = jnp.sum(item[None] * embeddings, axis=1)
  norm_item = jnp.linalg.norm(item)
  norm_all = jnp.linalg.norm(embeddings, axis=1)
  distances = (dot_products / (norm_item * norm_all))
  return distances

Let's define function that will find the most similar movies for one we specify

In [ ]:
def get_top_k_most_similar_movies(movie_name, embeddings, k):
  movie_id = movie_to_id_mapping[movie_name] # retrieve the movie_id
  distances = get_cos_sim_distances(embeddings[movie_id], embeddings) # calculate the distances between the movie and all others
  # Sort ascending and take the last k movie ID's (excluding the movie itself which will be the most similar i.e. angle = 0)
  # Then reverse the order so the most similar movie IDs are first
  return list(reversed(distances.argsort()[-k-1:-1].tolist()))

Let's call this method now and try it out on the second Avengers movie.

In [ ]:
similar_ids = get_top_k_most_similar_movies("Amazing Spider-Man, The (2012)", state.params['params']['item_embs']['embedding'], 5)
display_recommendations(similar_ids)

Try testing out with your own movies. To find the exact name of a particular movie, here's a little helper function that you can use.

In [ ]:

import ipywidgets as widgets
from IPython.display import display

# Function to search and display results
def search_dataframe(search_text):
    # Filter DataFrame based on search_text in the 'Name' column
    filtered_df = movies[movies['movie_title'].str.contains(search_text, case=False, na=False)]
    # Display the filtered DataFrame
    display(filtered_df)

# Create a text input widget
search_input = widgets.Text(
    value='',
    placeholder='Type a movie name...',
    description='Search:',
    disabled=False
)

# Create an output widget to display the results
output = widgets.Output()

# Define an event handler function for the search input widget
def on_search_change(change):
    with output:
        output.clear_output()  # Clear previous output
        search_text = change['new']
        search_dataframe(search_text)  # Search and display results

# Attach the event handler to the text input widget
search_input.observe(on_search_change, names='value')

# Display the input widget and the output
display(search_input, output)

We can also visualise our embeddings using TensorBoard's [Embedding Projector](https://www.tensorflow.org/tensorboard/tensorboard_projector_plugin) where we can see where in this space all our movie embeddings lie. Let's load the tool. Towards the bottom left you should see some tabs called UMAP, t-SNE, and PCA. The details of these techniques are out of scope but their basic idea is to cluster our embeddings into a lower dimensional space that we can visualise.

Try clicking around in the point clouds to see the similar movies around the point you select. You can also search for (and then click) for movies on the right. You can also reduce (or increase) the number of movies that are highlighted by tweaking tue `neighbours` parameter.

In [ ]:
# @title Launch Tensorboard Projector

import numpy as np
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter("./tensorboard/")
writer.add_embedding(np.asarray(state.params['params']['item_embs']['embedding']), metadata=list(id_to_movie_mapping.values()))

%load_ext tensorboard
%tensorboard --logdir ./tensorboard/

### **2.5 Making Recommendations**

Apart from finding similar movies based purely on their similarity to each other, we can also find the movies in the neighbourhood of users i.e. the movies most similar to a user. This may seem surprising since users and movies are different entities. But a way to think about this is that our embedding space is an abstract representation common to both in which we can measure similarity using a similarity metric.

In [ ]:
user_id = df['user_id'].sample(1).iloc[0] # select a random user
item_embs = state.params['params']['item_embs']['embedding'] # retrieve our trained movie embeddings
user_emb = state.params['params']['user_embs']['embedding'][user_id] # retrieve our trained user embedding
distances = get_cos_sim_distances(user_emb, item_embs) # calculate the distance between our user and all the movies
already_watched_movie_ids = df[df['user_id'] == user_id]['item_id'].tolist() # retrieve all the movies the user already watched
rec_count = 0
# Loop over the closest movies to the user that they have not yet watched
for id in reversed(distances.argsort().tolist()):
    if id not in already_watched_movie_ids and rec_count < 10:
        print(id_to_movie_mapping[id])
        rec_count += 1

Since our model is trained to predict ratings of movies, let's actually use it directly to predict the ratings a particular user would give to all the movies and recommend the movies they would have rated the highest.

In [ ]:
@jax.jit
def pred_step(state, batch):
  logits = state.apply_fn(state.params, batch)
  return logits

def create_eval_tfds_from_df(df: pd.DataFrame) -> tf.data.Dataset:
  eval_fields = {
      ('item_id', tf.int32),
      ('user_id', tf.int32),
  }
  tensor_slices = {
      field: tf.cast(df[field].values, dtype=field_type)
      for field, field_type in eval_fields
  }

  return tf.data.Dataset.from_tensor_slices(tensor_slices)

# prepare our data by mapping the same user id to all our movies
all_items = df['item_id'].unique()
eval_df = pd.DataFrame({'user_id': [user_id] * all_items.shape[0], 'item_id': all_items})
eval_ds = create_eval_tfds_from_df(eval_df).batch(eval_df.shape[0], drop_remainder=False)

# make the prediction on this data which returns the ratings
preds = pred_step(state, eval_ds.as_numpy_iterator().next())
eval_df['preds'] = pd.DataFrame(preds)

# Display the top 10 rated movies that the user has not yet watched
count = 0
for row in eval_df.sort_values('preds', ascending=False).itertuples():
  if row.item_id not in already_watched_movie_ids and count < 10:
    print(f"{row.preds:.2f}", id_to_movie_mapping[row.item_id])
    count += 1
  elif count == 10:
    break

### **2.6. Approximate Nearest Neighbours**

So far in this practical we've been performing an *exhaustive search* for the most similar movies i.e. we've been calcuating the distance between a movie and every other movie. What if we have millions of movies? And millions of users logging into our streaming service for which we need to make these calculations? This will indeed get very costly in both time and resources. We need a way to perform these calculations much more efficiently. This is where *Approximate Nearest Neighbours* comes in. The idea is we tradeoff a small degree of accuracy in our similarity search but in exchange gain an enormous amount of performance (orders of magnitude).

One such implementation of this idea, originally developed at Spotify, is called [*Approximate Nearest Neighbours Oh Yeah*](https://github.com/spotify/annoy) (yes, really) or more commonly known by it's acronym, *ANNOY*. The algorithm works by recursively splitting our embedding space by random hyperplanes, where each hyperplane is represented by a node in a tree, until there are at most $k$ items at every leaf node. In order to find the nearest neighbours for a specific embedding, the tree is traversed by calculating on which side of the hyperplane the point lies at every node and then returning the items at the final node. See the below image to get an idea of what the splitting looks like in a 2D space and the resulting tree.

<img src="../images/tree-full-K-1024x793.png" width="50%">

<img src="../images/tree-full-K-graphviz1-1024x404.png" width="60%">

Source: Erik Bernhardsson. [*Nearest neighbors and vector models*](https://erikbern.com/2015/10/01/nearest-neighbors-and-vector-models-part-2-how-to-search-in-high-dimensional-spaces.html). 2015.

There are a few more details to this algorithm which are out of scope for this prac but if you'd like to learn more about it's inner workings, read this excellent [blog post](https://erikbern.com/2015/10/01/nearest-neighbors-and-vector-models-part-2-how-to-search-in-high-dimensional-spaces.html), by the author himself of ANNOY, Erik Bernhardsson.

Let's now create our ANNOY index. This process is slow and can take up to 10 minutes to run (perhaps you can read the blog post while you wait).

We first create the `AnnoyIndex` where we specify the size of our embedding (the $d$ from earlier) and the distance metric. Here `angular` refers to the cosine similarity. We then loop over our embeddings, adding them to our index.

Next we build our index by specifying the number of trees it should make. In the earlier explanation, we showed one tree being built but in reality the algorithm actually makes use of many. More trees results in greater accuracy but require more memory. In practice you will specify the number of trees that either max out your memory or until your desired level of accuracy is reached.

In [ ]:
from annoy import AnnoyIndex
from tqdm import tqdm

t = AnnoyIndex(config.EMB_DIM, 'angular')
for i, emb in tqdm(enumerate(state.params['params']['item_embs']['embedding']), total=state.params['params']['item_embs']['embedding'].shape[0]):
    t.add_item(i, emb)

t.build(10)

Let's again look at the most similar movies to this Avengers movie. We do this by calling the `get_nns_by_item` method and specifying the movie we would like to look up, the `n` (approximate) nearest neighbours we'd like to return, and `search_k` which refers to the number of embeddings it will consider for the distance calculation. The tradeoff made is that the higher the `search_k`, the more accurate it will be but the slower the performance. So in practice this number will be chosen by making sure it's as high as possible to maximise accuracy while making sure the results are returned in a time under the threshold you set for your application.

In [ ]:
movie_id = movie_to_id_mapping["Avengers: Age of Ultron (2015)"]
for id in t.get_nns_by_item(movie_id, 5, search_k=10):
  print(id_to_movie_mapping[id])

Let's compare the results to the exact search we performed before. How does it compare? What if we tweak the `search_k` above?

In [ ]:
similar_ids = get_top_k_most_similar_movies("Avengers: Age of Ultron (2015)", state.params['params']['item_embs']['embedding'], 10)
for id in similar_ids:
  print(id_to_movie_mapping[id])

### **2.7. Types of Feedback**

To build recommender systems, there are two types of feedback we train our models against.
- **Explicit**: users specify whether or not they liked an item and sometimes even by how much. Examples of this are a thumbs up or thumbs down on a youtube video, review scores of a product, or even a movie rating!
- **Implicit**: the feedback is inferred based on how the users are interacting with the items. For example clicking an item, watching a video video, adding a product to your basket and/or purchasing.

The key differences between them is that explicit feedback is more sparse but gives you a higher confidence in to the users intent while implicit data is abundant but also noisier.

Usually both types of feedback are collected in the online services you use. Explicit feedback comes from the users naturally using the product (for e.g. reviewing the product) while the implicit data is usually coming from the telemetry infrastructure monitoring how users are interacting with your application. Depending on the maturity of this infrastructure, this can simply be logging what you're clicking on but can also include timing how long you've been on a page, the location your web requests are originating from, tracking exactly what content you've been exposed to, and even whether you simply hovered over an item and for how long. All of this data is logged and dumped into massive data stores to be later cleaned, analysed, and eventually trained on to create the recommender models.

Now we can actually answer the question posed in the title of this prac. These massive datasets comprised of both implicit and explicit feedback along with the large recommender models trained on them is how the recommendations can sometimes feels so good. So while your phone is not actually listening in on your conversation where you might have expressed some interest in those items, you more than likely also showed that interest implicitly by the simple usage of these internet platforms and this is all that was needed to figure out your preferences (especially when keeping in mind the thousands or even millions of people who might have interacted with the platform in similar ways). Now keep in mind how large these datasets and models can become. In this prac we built relatively small models on just 1MB of data and were still able to make plausible recommendations. They only get better when you scale them up.

Hopefully now that your mind is at ease that you're not being spied on, we hope that you're now not nervous about how much of your usage data is being logged. At least no you can make a more informed decision about whether or not the quality of the recommendations you get is worth this tradeoff. Keep in mind, the purpose of these systems is to help you find the content you're looking for and discover content you might not have otherwise found. If this is still not worth it for you, luckily regulations are catching up and that it may start to become mandatory for to allow users to opt out of personalised recommendations.

## Conclusion
**Summary:**

In this prac we introduced the general architecture of recommender systems and specifically focused on retrieving relevant items for users. We did this by demonstrating how Colaborative Filtering could be used for this task. Lastly we also described the type of data that's collected and how it's all crunched to construct the datasets used to build the recommender models.

**Next Steps:**

 - [Eugene Yan](https://eugeneyan.com/tag/recsys/) RecSys blog
 - [ACM RecSys YouTube](https://www.youtube.com/channel/UC2nEn-yNA1BtdDNWziphPGA)

**References:**

- Jeremy Howard & Silvain Gugger. [*Deep Learning for Coders with fastai and PyTorch*](https://www.oreilly.com/library/view/deep-learning-for/9781492045519/). 2020. O'Reilly.
- Kim Falk. [*Practical Recommender Systems*](https://www.manning.com/books/practical-recommender-systems). 2019. Manning.
- [*Google Recommendation Systems Course*](https://developers.google.com/machine-learning/recommendation).


For other practicals from the Deep Learning Indaba, please visit [here](https://github.com/deep-learning-indaba/indaba-pracs-2024).